# 01: SFT on GSM8K
Fine-tune Qwen2.5-0.5B-Instruct on GSM8K with answer format.
Run once, push to HF, then use in GRPO experiments.

Install dependencies

In [ ]:
!pip install -qUU git+https://github.com/tripathysagar/rlhf-gsm8k.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 51.3 MB/s eta 0:00:00


Import libraries

In [2]:
import torch
import regex as re
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig

Set training hyperparameters and HF repo target

In [3]:
config = dict(
    model_name="Qwen/Qwen2.5-0.5B-Instruct",
    lora_r=16,
    lora_alpha=16,
    epochs=3,
    lr=2e-4,
    batch_size=8,
    grad_accum=4,
    train_size=1024,
    hf_repo="tripathysagar/Qwen2.5-0.5B-GSM8K-SFT",  # change to your repo
)

Load tokenizer, model (bfloat16), and set up LoRA on all linear layers

In [4]:
tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
model = AutoModelForCausalLM.from_pretrained(
    config["model_name"], torch_dtype=torch.bfloat16, device_map="auto"
)

lora_config = LoraConfig(
    r=config["lora_r"],
    lora_alpha=config["lora_alpha"],
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Define the system prompt and helper functions to convert GSM8K examples into the chat format with a structured `The answer is: {number}.` ending

In [5]:
SYSTEM_PROMPT = (
    "You are a helpful math assistant. Solve the problem step by step, "
    "then give your final answer as a single number on the last line in exact format"
    """\nThe answer is: {number}."""
)

def clean_gold_answer(answer_text):
    """Strip <<...>> annotations and reformat with 'The answer is: N.' ending."""
    parts = answer_text.split("####")
    reasoning = parts[0].strip()
    final_num = parts[1].strip().replace(",", "") if len(parts) > 1 else ""
    reasoning = re.sub(r'<<.*?>>', '', reasoning)
    return f"{reasoning}\nThe answer is: {final_num}."


def make_sft_example(ex):
    cleaned = clean_gold_answer(ex['answer'])
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": ex["question"]},
        {"role": "assistant", "content": cleaned},
    ]
    return {"messages": messages}

Load GSM8K training split, shuffle, take a subset, and format as chat messages

In [6]:
ds = load_dataset("openai/gsm8k", "main", split="train")
ds = ds.shuffle(seed=42).select(range(config["train_size"]))
ds = ds.map(make_sft_example, remove_columns=ds.column_names)

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

Configure and run SFT training with LoRA

In [7]:
sft_config = SFTConfig(
    output_dir="sft_qwen_gsm8k",
    num_train_epochs=config["epochs"],
    per_device_train_batch_size=config["batch_size"],
    gradient_accumulation_steps=config["grad_accum"],
    learning_rate=config["lr"],
    logging_steps=10,
    save_steps=10,
    report_to='none',
    max_length=256,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    bf16=True,
    fp16=False,
    dataset_text_field=None,
    save_total_limit=2,
)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=ds,
    args=sft_config,
    peft_config=lora_config,
)

trainer.train()

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Tokenizing train dataset:   0%|          | 0/1024 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1024 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,1.445153
20,0.570653
30,0.368143
40,0.326823
50,0.306867
60,0.306866
70,0.303008
80,0.289909
90,0.283406


TrainOutput(global_step=96, training_loss=0.45599458118279773, metrics={'train_runtime': 186.1496, 'train_samples_per_second': 16.503, 'train_steps_per_second': 0.516, 'total_flos': 1695756329349120.0, 'train_loss': 0.45599458118279773})

Merge LoRA adapters back into the base model

In [8]:
model = trainer.model.merge_and_unload()

In [10]:
model.gradient_checkpointing_disable()

## Quick Eval

Formmating function for applaying the chat template.

In [12]:
def format_prompt(question):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [35]:
test_ds = load_dataset("openai/gsm8k", "main", split="test")
test_ds = test_ds.train_test_split(test_size=0.5, seed=42)
test_ds = test_ds["test"]
test_ds

Dataset({
    features: ['question', 'answer'],
    num_rows: 660
})

In [36]:
def extract_gold(ex):
    return {"answer": ex["answer"].split("####")[-1].strip()}

test_ds = test_ds.map(extract_gold, remove_columns=["answer"])
test_ds = test_ds.map(lambda x: {"question": format_prompt(x["question"])})
test_ds


Map:   0%|          | 0/660 [00:00<?, ? examples/s]

Map:   0%|          | 0/660 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 660
})

In [37]:
print(test_ds[0]['question'])
test_ds[0]['answer']

<|im_start|>system
You are a helpful math assistant. Solve the problem step by step, then give your final answer as a single number on the last line in exact format
The answer is: {number}.<|im_end|>
<|im_start|>user
Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now.<|im_end|>
<|im_start|>assistant



'109'

Fetching the answer from the response.

In [38]:
def extract_answer(response):
    try:
        matches = re.findall(r'The answer is[:\s]*(\-?\d[\d,]*\.?\d*)', response, re.IGNORECASE)
        if matches:
            return int(float(matches[-1].replace(",", ""))), True
        nums = re.findall(r'\-?\d[\d,]*\.?\d*', response)
        if nums:
            return int(float(nums[-1].replace(",", ""))), False
    except Exception as e:
        print(f"[extract_answer error] response={response[:80]!r} err={e}")
    return None, False


In [39]:
assert extract_answer("The answer is: 72.") == (72, True)
assert extract_answer("ans : 42") == (42, False)
assert extract_answer("no numbers here") == (None, False)

In [40]:
from tqdm import tqdm

def eval_math_accuracy (model, tokenizer, test_ds, batch_size=64):
    model.eval()
    model.gradient_checkpointing_disable()
    tokenizer.padding_side = "left"

    correct, total = 0, 0
    table_rows = []
    test_data = list(test_ds)

    for i in tqdm(range(0, len(test_data), batch_size), desc="Evaluating"):
        batch = test_data[i:i+batch_size]
        prompts = [format_prompt(ex["question"]) for ex in batch]
        golds = [int(ex["answer"].split("####")[-1].strip().replace(",", "")) for ex in batch]

        inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(model.device)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)

        for ids, gold_int in zip(out, golds):
            response = tokenizer.decode(ids[inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
            ans, has_fmt = extract_answer(response)
            is_correct = (ans == gold_int)
            correct += int(is_correct)
            total += 1
            table_rows.append(f"| {total} | {gold_int} | {ans} | {'✅' if is_correct else '❌'} | {response[:80].replace(chr(10), ' ')} |")

    print(f"\nAccuracy: {correct}/{total} = {correct/total:.2%}")

    model.train()
    tokenizer.padding_side = "right"
    return table_rows


In [41]:
perf_log = eval_math_accuracy (model, tokenizer, test_ds)

Evaluating: 100%|██████████| 11/11 [01:59<00:00, 10.85s/it]


Accuracy: 172/660 = 26.06%


In [48]:
from IPython.display import Markdown, display

header = "| # | Gold | Predicted | ✓ | Response |\n|---|---|---|---|---|"
display(Markdown(header + "\n" + "\n".join(perf_log)))

| # | Gold | Predicted | ✓ | Response |
|---|---|---|---|---|
| 1 | 109 | 188 | ❌ | The sum of Darrell and Allen’s ages is 162/162=98%. So Darrell is 162*7%=114 and |
| 2 | 89 | -84 | ❌ | Colleen trades 90/100 * 40 = 36 large stickers. She gets 3*36 = 108 large button |
| 3 | 13 | 13 | ✅ | Half the letters in Indras' name is 6/2 = 3. So, her sister's name has 3 + 4 = 7 |
| 4 | 5 | 5 | ✅ | Trey runs 10 + 4 = 14 laps. Shaelyn runs 14 / 2 = 7 laps. Quinn runs 7 - 2 = 5 l |
| 5 | 25 | 19 | ❌ | On the first street, there were 5 children chasing the truck. On the second stre |
| 6 | 452 | 1320 | ❌ | If she slept for 7 hours before waking up at 6 a.m., she spent 7 x 6 = 42 hours  |
| 7 | 43 | 42 | ❌ | After Arnold stole 12 of the bananas, there were 48 - 12 = 36 bananas left. Gunt |
| 8 | 34 | 14 | ❌ | Two dozen cups = 2 x 12 = 24 cups. Jonathan has 24/3 = 8 measuring spoons. He ha |
| 9 | 120 | 1973 | ❌ | If Martha's cat can run at a speed of 15 feet/second, it will take it 40 / 15 =  |
| 10 | 11 | 1 | ❌ | If the first plane flies to Greece for three quarters of its flights, it flies t |
| 11 | 34 | 27 | ❌ | Before the storm, Sally had 25+7+12=44 balloons. After the balloon burst, there  |
| 12 | 12 | 6 | ❌ | Kira had 3+5+6=14 fruits. Lola ate 2*4=8 pieces of fruit. So, there were 14-8 =  |
| 13 | 15 | 5 | ❌ | There were originally 10/3=3.33 liters of water in the orange drink. So there wa |
| 14 | 24 | 112 | ❌ | They watch 2 * 4 = 8 movies each weekend. So they watch 8/2 = 4 movies each day. |
| 15 | 105 | 65 | ❌ | Today, Mark will be able to complete 5*6=30 questions. So far, he has completed  |
| 16 | 120 | 120 | ✅ | If Amy made 20 more friends than Lily, then she made 20 + 50 = 70 friends. Toget |
| 17 | 17 | 4 | ❌ | Good = 25 - 1 - 20% of 25 - 2 = 25 - 25 - 5 - 2 = 4 The answer is: 4. |
| 18 | 36 | 36 | ✅ | An eraser costs $2 and an eraser costs $3. So 6 erasers will cost 6 x $2 = $12.  |
| 19 | 12 | 12 | ✅ | He uses 3 * 2 = 6 face masks per day. So he uses 6 * 2 = 12 face masks every 2 d |
| 20 | 45 | 45 | ✅ | Max bought 16 + 3 = 19 truck stamps. Max bought 19 - 9 = 10 rose stamps. Max bou |
| 21 | 45 | 45 | ✅ | Katelyn’s friend came to see 50 / 2 = 25 fairies. There were 50 + 25 = 75 fairie |
| 22 | 8 | 2 | ❌ | 1 foot = 12 inches. So, each piece of wire is 12/6=2 inches long. Thus, Tracy ob |
| 23 | 36 | 324 | ❌ | First find how many seats are in total in the auditorium: 4 rows * 18 seats/row  |
| 24 | 1 | 5 | ❌ | Ellen buys 3 salads at $6 each so that's 3*6 = $18. The yogurt cost $6/2= $3. So |
| 25 | 18 | 10 | ❌ | Sara scored 8 points and now has 3*8 = 24 points. Before she had 24-14= 10 point |
| 26 | 33 | 44 | ❌ | She spent 10+2*2+3*2=20 She spent 20+20*0.5=$10 She spent 20+20+2=42 She spent 4 |
| 27 | 1248 | 624 | ❌ | The third complex has 180 apartments because 60 / .1 = 600 and 600 + 60 = 660. H |
| 28 | 25 | 20 | ❌ | There are three ways to roll two even numbers in a row and four ways to roll a n |
| 29 | 1 | 22 | ❌ | Each pack costs 10*9=$90$ A 10% discount means Russell pays only 90%-10=80% for  |
| 30 | 32 | 32 | ✅ | Let x be the length of the shorter piece. Then the longer piece is 4x. Since the |
| 31 | 2 | 4 | ❌ | First find how much food the three cats eat per day: 60 grams/cat/day * 3 cats = |
| 32 | 30 | 30 | ✅ | First find how many of the elves quit after vomiting on them: 1/3*60 = 20 elves. |
| 33 | 4 | 4 | ✅ | Adam spent 7 * $0.5 = $3 on candy of type A. He spent 10 * $0.75 = $7.5 on candy |
| 34 | 54 | 594 | ❌ | First find the distance Elvis traveled traveling west: 5 miles/hour * 5 hours =  |
| 35 | 250 | 5280 | ❌ | If he stays for 8 hours, he will pay 8*1000=8000 pesos. After that, if he adds 7 |
| 36 | 324 | 45 | ❌ | First find the total cost of the original furniture: $129.00 + $15.00 = $144.00  |
| 37 | 129200 | 184285 | ❌ | If Juan's house is 30% less expensive than Herb's house, it means that if we let |
| 38 | 4400 | 5600 | ❌ | First find how many movies are in each series: 600 movies / 3 = 200 movies Then  |
| 39 | 70 | 1200 | ❌ | The total area of the rectangular yard is 20 * 15 = 300 square feet. Therefore,  |
| 40 | 276000 | 252000 | ❌ | He bought the cars for 12*20000=$240000 So he paid 240000*.1=24000 tax That mean |
| 41 | 108 | 12 | ❌ | There are 90 / 9 = 10 groups. Therefore, there are 10 * 3/5 = 6 groups that brou |
| 42 | 160 | 720 | ❌ | For the fish sub, the price is $40/foot * 12 inches = $480. For the six inch col |
| 43 | 90 | 90 | ✅ | First we need to find how much Johnny had before he put in more money - he start |
| 44 | 20 | 20 | ✅ | If the winner got 3/4 of the votes, then he got 3/4*80 = 60 votes. Since the win |
| 45 | 296 | 632 | ❌ | For the hotel, she spent 3 * $80 = $240. She also spent 7 * 10/100 * $80 = $56 o |
| 46 | 4800 | 5800 | ❌ | If the price of a diamond Jewell is $2000, the price of a gold jewell is (4/5)*$ |
| 47 | 30 | 26 | ❌ | His brother is 8 * 2 = 16 years old. His sister is 8 / 4 = 2 years old. Their to |
| 48 | 8 | 15 | ❌ | In one hour, the candle would melt 1*2 = 2 cm. So after 8 hours of burning, the  |
| 49 | 1920 | 192 | ❌ | First find the total volume of soil needed: 2 feet * 8 feet * 2 feet = 32 cubic  |
| 50 | 6 | 59 | ❌ | There were 20 * 3 = 60 pieces of bologna that Benny threw at the yellow balloons |
| 51 | 13 | 15 | ❌ | Let x represent Jerry's age five years ago. So now he is x+5 years old. And Jerr |
| 52 | 420 | 500 | ❌ | Let x be the amount taken by the first person. Then the second person takes x +  |
| 53 | 10 | -2 | ❌ | There was initially 24 / 6 = 4 liters of water. So there was 4 - 6 = -2 liters o |
| 54 | 12 | 16 | ❌ | One third of the 36 penguins that were sunning up are going to the ocean. That's |
| 55 | 50 | 45 | ❌ | On Monday, Robert collected 20 shells. On Tuesday, Kylie collected 2 * 20 = 40 s |
| 56 | 92 | 232 | ❌ | He has 2*24=48 bottles of water. So he'll have 2*70=140 guests over the party. T |
| 57 | 623 | 623 | ✅ | Alex weighs 4*125-2 = 498 pounds. Their combined weight is 125+498=623 pounds. T |
| 58 | 40 | 144 | ❌ | Courtney originally thought there were 48 people in attendance. Kelly said it wa |
| 59 | 17 | 31 | ❌ | First, we need to determine how far James swam. Since he swam 60% of the 20 mile |
| 60 | 940 | 950 | ❌ | In June, his expenditure was 60 - 500 = $450. So, his total expenditure for thos |
| 61 | 10 | 36 | ❌ | Each inch is 8/10 feet because there are 8 inches per foot. So the remora is 12  |
| 62 | 540 | 6 | ❌ | To find the total time Carrie's choir will be caroling, we need to add the durat |
| 63 | 110 | 110 | ✅ | He earned $30 * 7 = $210. So he has $210 - $100 = $110 left. The answer is: 110. |
| 64 | 20 | -10 | ❌ | Let c represent Wendy's current age and w represent Colin's current age. Accordi |
| 65 | 560 | 72 | ❌ | He goes to visit twice a month so that's 12*2 = 24 trips. It takes him 24 hours  |
| 66 | 24 | 24 | ✅ | Ben is two times older than Chris who is 4 so he is 2*4 = 8 years old. Caroline  |
| 67 | 22 | 22 | ✅ | Today, the alarm rings 4 times. On the second occasion, it rings 3 * 4 = 12 time |
| 68 | 4 | 4 | ✅ | If Rose bought 4 times the number of onions and potatoes Sophia bought, then Sop |
| 69 | 22 | 16 | ❌ | Let's assume that the number of votes in favor of the regulation is x. Since the |
| 70 | 64 | 14 | ❌ | Let's say Seth had 2x stickers. Luis took x/2 stickers from him, so he had 2x -  |
| 71 | 525 | 1750 | ❌ | Andy needs to lose 30 lbs so far this year, or 30 * 3500 = 105,000 calories. He  |
| 72 | 10 | 10 | ✅ | First find how many loaves are not sold in the morning: 60 loaves * 2/3 = 40 loa |
| 73 | 410 | 410 | ✅ | Zoey's total distance is 40*10 = 400 feet. Sydney's total distance is 35*12 = 42 |
| 74 | 140 | 140 | ✅ | He spends 7*2=14 hours playing video games. So he makes 14*$10=$140 Therefore, J |
| 75 | 720 | 720 | ✅ | He pays 4 * 15 = $60 for each car wash per month. So in a year, he pays 60 * 12  |
| 76 | 1050 | 8500 | ❌ | Since Julie operates the cash register exactly twice as fast as her colleague, s |
| 77 | 8 | 7 | ❌ | On Tuesday, Peyton’s cooking class lasted 3 * 1 = 3 hours. On Wednesday, Peyton  |
| 78 | 20 | 16 | ❌ | Jim had 27 + 5 = 32 seashells. Carrey had 32 / 2 = 16 seashells. They had a tota |
| 79 | 138 | 145 | ❌ | After one year, the price of carrots will be $120 * (1 + 5%) = $126. After two y |
| 80 | 175 | 140 | ❌ | Goldy gave away 20 x 3 = 60 kilograms of rice. She also gave away 20 x 4 = 80 ki |
| 81 | 64 | 80 | ❌ | If Colby completes 50 packages per hour, and he works for 8 hours, then he compl |
| 82 | 5600 | 8400 | ❌ | First find how many gallons of milk were unsold: 2 / 5 * 4000 gallons = 1600 gal |
| 83 | 720 | 720 | ✅ | Greta's paycheck includes $2400.00 and she puts 50% of it into her retirement ac |
| 84 | 312 | 2080 | ❌ | If Gary do laundry twice a week, he spends 2*20 = 40 gallons of water each week. |
| 85 | 60 | 40 | ❌ | In 8 weeks, Bailey gets $40 from her parents because 8 x 5 = 40. At the end of t |
| 86 | 450 | 490 | ❌ | The bakery has 7*70-40=490 loaves of bread. The answer is: 490. |
| 87 | 10000 | 600000 | ❌ | The total number of hours that all the patients see each day is 500*24=12000 hou |
| 88 | 43500 | 54000 | ❌ | First find how much the king cab upgrade costs: $30,000 / 3 = $10,000. Then subt |
| 89 | 150 | 32 | ❌ | Let x be the total number of cards Kenny had. He sold (x-1)/3 cards at $1.5 each |
| 90 | 9 | 9 | ✅ | First she got 3*1=3 dollars from the tooth fairy. Then she got .5*3=$1.50 for ea |
| 91 | 60 | 60 | ✅ | First find how many students took contemporary dance: 20 students / 100 * 20 = 4 |
| 92 | 24 | 24 | ✅ | Each friend received 2 + 1 = 3 water balloons. So there were 4 x 3 = 12 water ba |
| 93 | 539 | 552 | ❌ | First find how many sneakers he sold to his flash sale: 48 sneakers / 17 sneaker |
| 94 | 18 | 5 | ❌ | First find how many toys 2 workers make in one hour: 6 toys/hour + 4 toys/hour = |
| 95 | 14 | 8 | ❌ | If the club ordered 20 pizzas and ate 70%, they have 10% left. If the football t |
| 96 | 60 | 60 | ✅ | First find how far the bus traveled: 60 miles/hour * 5 hours = 300 miles. Then f |
| 97 | 1080 | 270 | ❌ | Before the T-Rex ate, there were 270 x 2 = 540 kilograms of meat. Therefore, the |
| 98 | 291 | 16 | ❌ | First find the total number of bouquets Faraday sold: $26 / $56 = 0.4375 bouquet |
| 99 | 70 | 70 | ✅ | Carmen spent 10*3=30 minutes solving crossword puzzles. She spent 5*8=40 minutes |
| 100 | 63 | 63 | ✅ | His brother eats 18 x 1/2 = 9 pretzels a day. So he eats 9 x 7 = 63 pretzels in  |
| 101 | 26 | 26 | ✅ | If the crocodile grows 8 inches in 4 years, then it grows 2 inches per year. In  |
| 102 | 750 | 1 | ❌ | To determine how many telephone calls Jason needs to make to sell 15 cars and ea |
| 103 | 54 | 56 | ❌ | If there were 10 cars pulled on each day for the first three days, then there we |
| 104 | 96 | 96 | ✅ | Let x be the number of strawberry cubes. Then, the number of blueberry cubes is  |
| 105 | 67 | 67 | ✅ | Cho hiked 14 x 8 = 112 kilometers. Chloe hiked 9 x 5 = 45 kilometers. Therefore, |
| 106 | 32 | 48 | ❌ | First find how many pieces of each fruit are in a crate: 12 apples * 2 = 24 appl |
| 107 | 8 | 8 | ✅ | Each box of chocolate has 64/4 = 16 chocolate bars. Each box of chocolate has 56 |
| 108 | 20 | 20 | ✅ | For the first 30 days, Chester eats 3 eggs per day so he eats 3*30=90 eggs. Afte |
| 109 | 76 | 24 | ❌ | If there are 4 intersections with 6 poles each, then there are 4 * 6 = 24 poles. |
| 110 | 21 | -11 | ❌ | First find how much Dean spent on the toy cars: $2/car * 6 cars = $12. Then find |
| 111 | 600 | 600 | ✅ | Each rose has 8 thorns x 25 roses = 200 thorns. So, Dan will have 3 rose bushes  |
| 112 | 16 | 2048 | ❌ | Julia takes 20/16 = 1.25 minutes per foot. She rows at a rate of 20/1.25 = 16 fe |
| 113 | 4 | 39 | ❌ | First we need to convert the distances to feet because Steve's speed is given in |
| 114 | 104 | 56 | ❌ | Last week, Meredith wrote 1 * (2/5) = 2 articles on Tuesday. So, she wrote a tot |
| 115 | 10 | 20 | ❌ | If Julia used 3 spoons during the stew making process, and her husband gave her  |
| 116 | 75 | 750 | ❌ | He gets 10*10=100 bottle caps per day. So he makes 100*.25=$25 every day from bo |
| 117 | 6 | 15 | ❌ | She ate 1 / 8 = 10% of the apples. Her sister ate 1 + 1 = 2 slices. Her brother  |
| 118 | 20000 | 25000 | ❌ | The first year the company spends $15000. For the second year, they spend 1/3 *  |
| 119 | 800 | 10000 | ❌ | He bought 1/2*40=20 red ties. That means he bought 20/2=10 red ties. So he bough |
| 120 | 45 | 125 | ❌ | Annabelle's total earnings from her first job are 20*10=200 dollars. Her total e |
| 121 | 288 | 288 | ✅ | Jolene filled up 10 * 30 = 300 balloons. They had 300 - 12 = 288 balloons left.  |
| 122 | 40 | 480 | ❌ | On Thursday, the mechanic earns 6 * $60 = $360. On Friday, he does not earn mone |
| 123 | 665 | 665 | ✅ | To solve this problem, we need to follow these steps: Step 1: Calculate the tota |
| 124 | 200 | 200 | ✅ | They have 30 seats because 1+7=8 They saved 6 people for their friends because 1 |
| 125 | 20 | 20 | ✅ | There are 20 - 12 - 4 = 4 semi-automatic cars. So, 4/20 * 100% = 20% The answer  |
| 126 | 20 | 20 | ✅ | The discount for the jeans is .25*40=$10. So after the discount, the jeans cost  |
| 127 | 24 | 47 | ❌ | Let's assume there are x students in each class. Since each class has the same a |
| 128 | 2 | 0 | ❌ | Igor gathered 1/2 * 16 = 8 test tubes. He gathered 14 + 2 = 16 Petri dishes. He  |
| 129 | 8 | 8 | ✅ | After saving $11 from his allowance, he has 21 + 11 = 32 dollars. He then spends |
| 130 | 20 | 240 | ❌ | First find how long Movie B is by adding 5 to the length of Movie C: 5 = 1.25 *  |
| 131 | 576 | 16 | ❌ | If 6 potatoes make 36 hash browns, we can find the number of hash browns one pot |
| 132 | 374 | 446 | ❌ | The bird flew 30*10=300 miles. The bird flew 18*2=36 miles. And the bird flew 22 |
| 133 | 80 | 2046 | ❌ | First find how many total yards Blake ran: 15 * 150 = 2250 yards Then add that t |
| 134 | 15400 | 7200 | ❌ | The hotel was 20/100*2000=400 extra dollars. So the hotel was 2000+400=2400 extr |
| 135 | 60 | 0 | ❌ | Michael travels at a rate of 50 kilometers per hour. He takes breaks every 4 hou |
| 136 | 11 | 8 | ❌ | Initially there were 2 + 1 = 3 apples. After throwing the first apple Newton had |
| 137 | 40 | 130 | ❌ | First find how many hours Jill worked: 2 + 1 = 3 hours. Then double that amount  |
| 138 | 8 | 8 | ✅ | When John was 32, his sister Dora was 12 because 32 - 3 = 28 When Dora turned 12 |
| 139 | 47 | 40 | ❌ | Larry has 3*3=9 dogs. Larry has 9-2 = 7 rabbits. Larry has 3*7=21 fish. Larry ha |
| 140 | 4 | 4 | ✅ | First find the total number of pets Melissa needs to groom: 8 + 5 + 3 + 12 = 28  |
| 141 | 140 | 160 | ❌ | He plants 90 + (90-40) = 160 flower in all. The answer is: 160. |
| 142 | 8 | 90 | ❌ | First find the total amount of time that has passed 15 days: 15 days * 2 = 30 da |
| 143 | 142 | 142 | ✅ | He gets 22 x 5 = 110 tomatoes. He also gets 4 x 8 = 32 eggplants. Therefore, he  |
| 144 | 575 | 800 | ❌ | Iris spent 1/4 * $400 = $100 for a manicure. She spent 3/4 * $400 = $300 for a p |
| 145 | 68 | 63 | ❌ | Since Mary is two years younger than Jessa, she is 20 - 2 = 18 years old. Joan i |
| 146 | 16 | 6 | ❌ | First find how many yards of material Jo needs to make one small mask: 2 yards / |
| 147 | 736 | 625 | ❌ | On Tuesday they delivered 1/5*425-425=65 letters. On Wednesday they delivered 2* |
| 148 | 27 | 27 | ✅ | Cindy had 2 x 5 = 10 blue eggs. She had 10 - 1 = 9 pink eggs. She had (9/3) x 1  |
| 149 | 68 | 8 | ❌ | If Katerina uses 8 gemstones for every topaz, she will need 8 x 4 = 32 pieces of |
| 150 | 32 | 31 | ❌ | Each time they go to the class together, it takes them 2*4=8 hours. So they spen |
| 151 | 30 | 8 | ❌ | Let c be the number of children and a be the number of adults. The total cost fo |
| 152 | 480 | 480 | ✅ | For one day, it would cost $25 + $1.50 x 10 = $40. In three days, it would cost  |
| 153 | 110 | 160 | ❌ | At the first stop, 20 + 120 = 140 people boarded the train. At the second stop,  |
| 154 | 6 | 5 | ❌ | Dolly reads 2 books + 1 book = 3 books. Pandora reads 1 book + 1 book = 2 books. |
| 155 | 93 | 75 | ❌ | On Monday Alice sees 3 clouds. Tuesday, Wednesday and Thursday both see 3 clouds |
| 156 | 10 | 10 | ✅ | First find how fast Bill digs through clay: 4 ft / hour * 0.5 = 2 ft / hour Then |
| 157 | 14 | 10 | ❌ | There were 2 x 6 = 12 people from the first type of family. There were 3 x 4 = 1 |
| 158 | 10 | 120 | ❌ | First find the total number of washes the 1 pump gives her: $120 / 120 = 10$ was |
| 159 | 40 | 85 | ❌ | Bobby has 5 * (20 - 5) + 5 = 85 total video games. The answer is: 85. |
| 160 | 4 | 4 | ✅ | Each pack contains 6*10=60 cookies. So, Carl receives 60*0.1=$6 from each pack o |
| 161 | 6250 | 625 | ❌ | First find how many dollars the students contributed to the charity: $175,000 -  |
| 162 | 5 | 1 | ❌ | Ray gave away 4 * 5 = 20 lollipops to his friends. Therefore, he received 25 - 2 |
| 163 | 200 | 140 | ❌ | Allan had 2 * 40 = 80 cannolis. After he bought 60 more, Allan has 80 + 60 = 140 |
| 164 | 85 | 360 | ❌ | Let's say Maggie has x dollars. Riza has 1/3 * x = x/3 dollars. Together they ha |
| 165 | 100 | -350 | ❌ | Sarah does 400/4 = 100 pounds of laundry. Raymond does 100/2 = 50 pounds of laun |
| 166 | 20 | 14 | ❌ | If it takes 240 minutes to break in the shoes and there are 7 days in a week, th |
| 167 | 48 | 58 | ❌ | First, let's find out how many pieces were left after the first two days. He sta |
| 168 | 3000 | 6000 | ❌ | First, we need to calculate how far Stanley traveled down from his initial posit |
| 169 | 41 | 42 | ❌ | First find how many correct answers Amy gets for each type of question: she gets |
| 170 | 15 | 15 | ✅ | If Lindsay washed 10 pairs of socks, then there were 2 * 10 = 20 complete pairs  |
| 171 | 48 | 0 | ❌ | First find the total number of calories in one bag of chips: 250 calories/servin |
| 172 | 15 | 66 | ❌ | Phineas will be traveling 200/2=100 yards on land. So, he will travel 100*20=200 |
| 173 | 106 | 2 | ❌ | Benedict has already paid $100 * .02 = $2 for each month. He still owed $100 - $ |
| 174 | 14 | 8 | ❌ | Julia and Nadine spent $11 + $0.5 x 3 = $11 + $1.5 = $12.5. They spent a total o |
| 175 | 34 | 4 | ❌ | There are 84/12=7 groups of people. Each group has 9-5=4 men. If 20 women alight |
| 176 | 66 | 28 | ❌ | She gets 1*6=6 issues from the first magazine. Her second magazine gives her 6*4 |
| 177 | 20 | 260 | ❌ | If the first car is traveling at an average speed of 60 miles per hour and the s |
| 178 | 42 | 63 | ❌ | If there was a ratio of 7:13 for sugar to water, it means that every time she ma |
| 179 | 30 | 16200 | ❌ | John has to go to 3 more classes because 6 - 2 = 4. He spends 120 minutes per cl |
| 180 | 15 | 15 | ✅ | There are 10*5=50 boxes of stuff that were donated. So there are 50+10=60 boxes  |
| 181 | 16 | 100 | ❌ | He spent 2+3=5 dollars on milk. He spent 3+3=6 dollars on eggs. He spent 3+3=6 d |
| 182 | 251 | 5625 | ❌ | First find how many guests are expected by multiplying the number of people invi |
| 183 | 175 | 150 | ❌ | The first friend made 20+40 = 60 presses. The second friend made 60* (1/4) = 15  |
| 184 | 32 | 32 | ✅ | There are 12 / 2 = 6 male lions. So there are 12 + 6 + 14 = 32 lions in the encl |
| 185 | 50 | 35 | ❌ | She runs at 10mph * 3 = 30 miles. She runs at 5mph * h = 5h miles. So in total,  |
| 186 | 330000 | 98000 | ❌ | He paid $5000/month x 12 months = $60,000 for rentals. He spent $60,000 / 2 = $3 |
| 187 | 45 | 50 | ❌ | Beatrice had 2*14=28 roll-ups. Marcell had 3*24=72 roll-ups. Together, they had  |
| 188 | 12 | 2 | ❌ | Since there are 12 inches in a foot, Mrs. Jameson’s bamboo grows up to 30*12 = 3 |
| 189 | 18 | 3 | ❌ | If the cost of the pizza is $15 and she wants to give a tip of $15 / 5 = $3. The |
| 190 | 75 | 75 | ✅ | He eats 2 * 30 = 60 yogurts over 30 days. That means he spends 60 / 4 * $5 = $75 |
| 191 | 21 | 43 | ❌ | Andrea gave her brother $4 so that was $36 - $4 = $32. That means Andrea saved $ |
| 192 | 240 | 240 | ✅ | If they picked 60 oranges and Stetson ate 2 / 5 of them, then he ate 60 x 2 / 5  |
| 193 | 55 | 61 | ❌ | First find how much Alison paid for the pair of running shorts: $5 + $10 = $15 T |
| 194 | 490 | 290 | ❌ | If George has 200 bananas and he has 45% more pears than bananas, then he has 20 |
| 195 | 20 | 354 | ❌ | He reads 45*200=9000 words So it takes him 60/10=6 hours That means it took him  |
| 196 | 81 | 9 | ❌ | First find how many rounds of calls there are in total: 4 - 1 = 3 rounds. Then m |
| 197 | 250 | 350 | ❌ | If there were 100 customers on the first day and 50 more customers on the second |
| 198 | 800 | 800 | ✅ | The price after a 20% discount on a $1000 laptop is 1000 - (1000 * 20/100) = $80 |
| 199 | 4 | 7 | ❌ | First find how many holes Nate fills each day: 14 days * 9 holes/day = 126 holes |
| 200 | 14000 | 30000 | ❌ | In the first week, Boris produced 8000/2=4000 bars of chocolate. So in the secon |
| 201 | 4 | 17 | ❌ | Let x be the number of orange fish Bob bought. Then he bought x+17 white fish be |
| 202 | 250 | 260 | ❌ | He pays $500 x 10 = $5000 per month. So he earns $5000 - $25 x 10 = $5000 - $250 |
| 203 | 366 | 366 | ✅ | In the second month, there were 60 * 3 = 180 downloads. After that, it reduced t |
| 204 | 100 | 100 | ✅ | There were initially 172 passengers on the train. At the first stop, 47 people g |
| 205 | 7 | 7 | ✅ | Chips cost $20 - (2*2) = $4.00 Soda costs $1.00. So far he has spent $4 + $8 + $ |
| 206 | 54 | 8 | ❌ | If there are 4 people in total (Hejoe plus his teacher and his 25 classmates), t |
| 207 | 70 | 30 | ❌ | If there were seven buttons with two holes, they made a total of 7*2=14 holes. T |
| 208 | 273 | 273 | ✅ | Mariah used 1/4 * 364 = 91 yards of yarn. Her grandma used 1/2 * 364 = 182 yards |
| 209 | 1200 | 1680 | ❌ | Each room has two sheets + one comforter = three items. There are twice as many  |
| 210 | 75 | 90 | ❌ | Susan works from 8 a.m. to 11 a.m., which means she spends 3 hours doing the wor |
| 211 | 335 | 305 | ❌ | First find the total cost of the leggings: $100 + $5 = $105. Then find the total |
| 212 | 300 | 300 | ✅ | The total weight of the onions is 4 * 50 = 200 pounds. So the chef spent 200 * 1 |
| 213 | 1 | 6 | ❌ | Martha has 5+7=12 notebooks. Joseph has 12-3=9 notebooks. She has 9-3=6 more tha |
| 214 | 300 | 300 | ✅ | She spent 500*.2=$100 on clothes. So she was left with 500-100=400 dollars after |
| 215 | 398 | 408 | ❌ | If his brother’s playset has 80 toys and he needs 2 more toys to have enough for |
| 216 | 27 | 27 | ✅ | If Trinity sold the magazines for 11/8 times the price she bought them for, she  |
| 217 | 43200 | 144 | ❌ | A dwarf can mine 12*2 = 24 pounds of ore with an iron pickaxe. A dwarf can mine  |
| 218 | 130 | 130 | ✅ | First find how many total hours Lloyd tutored: 5 + 8 = 13 hours. Then multiply t |
| 219 | 110 | 110 | ✅ | If there were 70 gold coins and there were 30 more gold coins than silver coins, |
| 220 | 6 | 1 | ❌ | The round trip took 3 hours because 200+240=440 The round trip took 1 hour becau |
| 221 | 30 | 30 | ✅ | If her mom got 20 apples, and Jenna picked half as many, she picked 1/2 * 20 = 1 |
| 222 | 40 | 20 | ❌ | Luri has 3 * 20 = 60 songs. So Luri needs 100 - 60 = 40 more songs to fill her p |
| 223 | 80 | 45 | ❌ | After two months, there were 2 * 10 = 20 pencils that were used. Therefore, ther |
| 224 | 18 | 16 | ❌ | It took Dior 4+2 = 6 hours to complete her laundry. This means it took her 6+4 = |
| 225 | 5 | 5 | ✅ | First find the total cost of the pencils: $0.50/pencil * 6 pencils = $3.00 Then  |
| 226 | 7 | 7 | ✅ | The cost of three apples is 3 * $1.50 = $4.50. The cost of five oranges is 5 * $ |
| 227 | 250 | 184 | ❌ | She gets paid 20*8=$160 for 8 hours of work. She should get paid 160*.15=24 extr |
| 228 | 300 | 300 | ✅ | In one year Tim gets 6*5=30 lemons. So in a decade, he gets 30*10=300 lemons. Th |
| 229 | 22 | 18 | ❌ | Mitchell bought 2 * 55 = 110 chips. Each person will get 110 / 6 = 18.3 chips. T |
| 230 | 20 | 10 | ❌ | There are 32 - 9 - 6 - 3 = 10 kids who are not allergic to anything. The answer  |
| 231 | 3 | 77 | ❌ | Beck originally had 100 - 23 = 77 apples. If he receives 10 apples from Boris, h |
| 232 | 65960 | 9300 | ❌ | First find how many dollars it would cost to repave Monica's street: $194/meter  |
| 233 | 20 | 20 | ✅ | Half of the books are written in English, so there are 50 / 2 = 25 English-writt |
| 234 | 2125 | 3625 | ❌ | The second box has 3*500=1500 pieces. So in total there are 500+1500+500 = 2500  |
| 235 | 1100 | -2956 | ❌ | He bought 3*1800=$<<3*1800=5400>>5400 worth of magic cards. There were 30+20+100 |
| 236 | 14 | 26 | ❌ | In the first game, Julie scored 14 + 4 = 18 points. In the second game, Julie sc |
| 237 | 3 | 23 | ❌ | The paper costs 3*2+1=$7 So it costs 7*3=21 dollars Thus, the total cost is 2+21 |
| 238 | 33 | 33 | ✅ | Carlton has 13 + 7 = 20 players. Together, they have 20 + 13 = 33 players. The a |
| 239 | 2400 | 2400 | ✅ | First find how many square feet of flooring you need to cover your kitchen floor |
| 240 | 28 | 407 | ❌ | Let's use A for the number of books Anne has. Sofie has 25 + 12 = 37 books. Anne |
| 241 | 180000 | 43000 | ❌ | In one week, Tyson ran 5000 x 2 = 10000 meters. So in a month, he would have run |
| 242 | 20 | 0 | ❌ | The total mileage of all trips is 75 + 100 + (100+50) = 225 miles. So, the train |
| 243 | 88 | 88 | ✅ | Ms. Thompson gets 66 messages. Mr. Yu gets 1/3 * 66 = 22 messages. So James deli |
| 244 | 75 | 38 | ❌ | First find how many times Pauline goes to the museum after 5 years: 3 times/year |
| 245 | 5 | 3 | ❌ | In one hour he travels 20 miles because 1*10=10 and 10+6=16 So in two hours he t |
| 246 | 25 | 45 | ❌ | Let x be the time that Elaina's usual concert runs. Since she makes the concert  |
| 247 | 36 | 127 | ❌ | In five weeks, Carol has 20 signatures / week * 5 weeks = 100 signatures. To mak |
| 248 | 1210 | 1210 | ✅ | There are 80 spiders * 8 legs = 640 legs from the spiders. There are 90 insects  |
| 249 | 13 | 6 | ❌ | Walt's distance on Tuesday was 6 * 4 = 24 miles. His total distance for the firs |
| 250 | 260 | 11 | ❌ | He will need $3500 total because 2 times 15 is 30 and 3500 minus 30 is 3470. He  |
| 251 | 56 | -84 | ❌ | There are 36 candles total because 8 x 4 = 32. There are 72 flashlights because  |
| 252 | 48 | 29 | ❌ | Each school sends a team with 5+1=6 people. So, there are 4*6 = 24 people from t |
| 253 | 70 | 35 | ❌ | There are 10/2=5 robots in the changing room. There are 20/2=10 footballs in the |
| 254 | 14 | 2 | ❌ | Samantha was 31-6 = 25 years old. If Raymond's son was born when Samantha was 25 |
| 255 | 9 | 63 | ❌ | Frederick has 10 * 100/100 = 10 plants. Toni has 10 + 60 = 70 plants. Shondra ha |
| 256 | 20 | 10 | ❌ | On the first stop, there were 20 + 40 = 60 people on the bus. At the second stat |
| 257 | 3 | 3 | ✅ | Zack's locker is half as big as Timothy's so it is 24 / 2 = 12 cubic inches. Pet |
| 258 | 440 | 320 | ❌ | For the 4 hours spent teaching to a novice, Charlie's hourly rate is $80/4 = $20 |
| 259 | 576 | 198 | ❌ | First find how many tiles are needed per sq ft: 24 tiles/sq ft * 2/3 = 16 tiles/ |
| 260 | 2 | 2 | ✅ | First find the total cost of the milk: $3/ pack * 5 packs = $15. Then add that t |
| 261 | 18 | 56 | ❌ | She has 16*3=48 breakfast eggs. She has 16*4=64 baking eggs. So she has 48+64=11 |
| 262 | 188 | 188 | ✅ | The first book is 31 mm thick. The second book is 50 mm thick. The third book is |
| 263 | 8 | 8 | ✅ | The total cost of the shirt and shorts is $25 + $35 = $60. With a 30% discount,  |
| 264 | 90 | 1 | ❌ | First find the total weight of all the M&Ms: 3 large bags * 10 ounces/large bag  |
| 265 | 40 | 100 | ❌ | First find the total number of spools of thread Candy has: 15 light blue + 45 da |
| 266 | 42 | 70 | ❌ | Eva = $20.00 Anthony = $20 + $10 = $30.00 Roy = $30 + $40 = $70.00 The answer is |
| 267 | 675 | 1050 | ❌ | If Ezra reads 300 books in an hour, he would read 2*300=600 books in one hour. A |
| 268 | 1218 | 1218 | ✅ | Mamou distributed a total of 64 + 30 + 48 = 142 meals. She had 1360 - 142 = 1218 |
| 269 | 1260 | 1260 | ✅ | He gets paid for 30*5=150 tasks per day. So, he makes 150*$1.2=$180 a day. In a  |
| 270 | 13 | 25 | ❌ | Each bottle of soda costs $21/7 = $3. Each bottle of water costs $8/4 = $2. So e |
| 271 | 22 | 22 | ✅ | On Thursday, he picked 6 - 4 = 2 bananas. On Friday, he picked 3 * 4 = 12 banana |
| 272 | 310 | 26 | ❌ | There are 3 months from January to March. If there are 10 students on campus at  |
| 273 | 105 | 135 | ❌ | Elijah’s dog weighs 1/4*60=15 pounds. Kory’s dog weighs 60*2 = 120 pounds. Elija |
| 274 | 100 | 40 | ❌ | If Mark's body weight decreased by 10 pounds every month, it means that if we ad |
| 275 | 19 | 85 | ❌ | On the second day, they used 10 less than 38, which is 38 - 10 = 28. So, they us |
| 276 | 40 | 40 | ✅ | If Mico's age is x and Marco's age is y, we can say that x + y = 20. In 10 years |
| 277 | 18000 | 18000 | ✅ | Jessica pays $300 + $200 = $500 for her student loans. She also pays $500 + $500 |
| 278 | 7 | 76 | ❌ | In the first 8 weeks, Jane has saved a total of $4 x 8 = $32. It will take her $ |
| 279 | 75 | 150 | ❌ | There are 50 ants * 2 ants/ant = 100 bugs. Insects in the garden = Bugs + Ants = |
| 280 | 14 | 15 | ❌ | Each person trades 2 old records for 1 new record so each person ends up with 2  |
| 281 | 16 | 7 | ❌ | The car in the slow lane was traveling at 30 miles per hour because 60/2=30. The |
| 282 | 15 | 15 | ✅ | Annika spent 50/2 = $25 on food and snacks. She also spent $10 on rides, so she  |
| 283 | 3 | 4 | ❌ | First find how much taller the beanstalk is after 1 day: 4 feet/day * 2 = 8 feet |
| 284 | 1800 | 2520 | ❌ | In one hour, Marissa can make 120 x (3/4) = 90 pounds of chocolates. So, they ca |
| 285 | 140 | 30 | ❌ | It will take 10 x 3 = 30 minutes for the fog bank to cover the whole city. The a |
| 286 | 17000 | 13000 | ❌ | Mr. Josue's total capital after getting money from both banks is 5000 + 2*4000 = |
| 287 | 45 | 45 | ✅ | If Aliya is 3 years old and Shawna is three times as old as Aliya, then she is 3 |
| 288 | 3200 | 1500 | ❌ | In one week, he made 10 * $10 = $100. So in four weeks, he would have earned 4 * |
| 289 | 160 | 160 | ✅ | First find the total number of flowers in the garden: 10 rows * 20 flowers/row = |
| 290 | 27 | 19 | ❌ | Each of the grandparents have 3 x 2 = 6 great-grandchildren. Thus, there are 6 x |
| 291 | 12 | 8 | ❌ | Initially there were 50-4*5=20 cats. After that 20*3/5=12 cats ran after a mouse |
| 292 | 320 | 240 | ❌ | After the first month, Elaine has 3 * 20 = 60 Pokemon. In the second month, she  |
| 293 | 36 | 40 | ❌ | Josh spent 4/7 * 8 = 3.9292929292929292 weeks in workouts. He spent 5 + 6 = 11 h |
| 294 | 24 | 7 | ❌ | If John is 28 and he is twice as old as Jim, that means Jim is 14/2 = 7 years ol |
| 295 | 1 | 8 | ❌ | Doctor Jones spends 20+30=50 minutes per hour updating his records. He spends 50 |
| 296 | 27 | 18 | ❌ | Daniel has to pay for five notebooks = 5 x $1.50 = $7.50 He has to pay for two p |
| 297 | 240 | 880 | ❌ | Dylan's investment = 2/5 * $1200 = $480. Total amount before Frances' investment |
| 298 | -3 | 4 | ❌ | The temperature decreased from 2 to -6 which means it became 0 degrees after tha |
| 299 | 19 | 22 | ❌ | If three customers were kicked out for shoplifting, they were kicked out for 3*5 |
| 300 | 694 | 429 | ❌ | First find the total cost of the donuts: 3 dozen * $68/dozen = $204 Then find th |
| 301 | 215 | 155 | ❌ | First convert all times into same unit of measurement for easier calculation. Th |
| 302 | 32 | 32 | ✅ | Ali has 4*$10=$40 from his first type of bill. He also has 6*$20=$120 from his s |
| 303 | 12 | 12 | ✅ | Freda used 16/2 = 8 cans of tomatoes. Each can contained 3/2 = 1.5 tomatoes. So, |
| 304 | 34 | 34 | ✅ | He spent 4 x $1.5 = $6 for the pens. He also spent 2 x $4 = $8 for the notebooks |
| 305 | 10 | 15 | ❌ | Before he gave away some shoe boxes, there were 7 + 9 = 16 shoe boxes. He used 3 |
| 306 | 25 | -61 | ❌ | First find the cost of the two vaccine doses: $20/vaccine * 2 doses = $40. Then  |
| 307 | 2400 | 730 | ❌ | If John bought 10 bottles at $600/10= $60/bottle. He spent 2*$60=$120 on cognac. |
| 308 | 16 | 16 | ✅ | Before Susan divided the cookies, she had 100/6 = 16.67 cookies left over. Her h |
| 309 | 31 | 55 | ❌ | If each new pen requires two quarts of ink, Ram needs 25/2 = 12.5 pens. So, he w |
| 310 | 25 | 100 | ❌ | She gets one pair at no cost because it’s 4/4 = 1 pair. So she only pays for 3 p |
| 311 | 80 | 80 | ✅ | Let L be the original length of the dress in cm. L + 0.5L + 20 = 140 1.5L = 120  |
| 312 | 9 | 18 | ❌ | There are 26 + 46 = 72 pencils. So, they have a total of 3 + 1 = 4 boxes of penc |
| 313 | 1430 | 2230 | ❌ | First find how much Janet paid for the brooch before insurance was added: $500 + |
| 314 | 2 | 10 | ❌ | Each cow produces 5 liters per day. So, three cows will produce 5 x 3 = 15 liter |
| 315 | -10 | -6 | ❌ | The total sum of the temperatures from the three countries is -48 + 21 + (-3) =  |
| 316 | 17 | 17 | ✅ | First find the total cost of the hamburgers: $4/hamburger * 5 hamburgers = $20 T |
| 317 | 73 | 180 | ❌ | Rick killed 3 * 15 = 45 cougars. So he killed 45 * 3 = 135 wolves. Therefore, Ri |
| 318 | 14000 | 56000 | ❌ | On the first day, the amount of water passing through the river doubles to 4000* |
| 319 | 880 | 208 | ❌ | A watermelon costs $3 * 4 = $12. An orange costs $12 - 5 = $7. Dillon spends $12 |
| 320 | 31 | 18 | ❌ | Carl used 2 slices per sandwich so for 5 sandwiches, he used 2 x 5 = 10 slices o |
| 321 | 10 | 10 | ✅ | First find how many pencils Anthony gave to Brandon: 1/2*50=25 pencils. Then sub |
| 322 | 104 | 105 | ❌ | Let's say the price of the boots is x dollars. Since the boots together cost 5 d |
| 323 | 13 | 27 | ❌ | Jasmine had 15 + 7 = 22 stickers. She got another 5 stickers so she has 22 + 5 = |
| 324 | 38 | 17 | ❌ | First find the total number of cards Erica’s father gave her: 2 boxes * 15 cards |
| 325 | 70000 | -15000 | ❌ | First find the total cost of putting in repairs: $80,000 + $50,000 = $130,000 Th |
| 326 | 3 | 6 | ❌ | First find the total amount of pizza that Jenny gave away: 1 / 3 * 12 = 4 slices |
| 327 | 57500 | 6020 | ❌ | She makes $20 x 35 = $700 as a teacher. She also makes $30 x 15 = $450 as a chee |
| 328 | 4 | 4 | ✅ | He lost 20-80%=12% of the matches. So he lost 20*1/5=4 people. The answer is: 4. |
| 329 | 1400 | 1400 | ✅ | First find how many bananas the zookeeper needs for each group of animals: 200 + |
| 330 | 92 | 156 | ❌ | Double the number of books in the first row is 20 x 2 = 40 books. Twice the numb |
| 331 | 3 | 0 | ❌ | First find the amount of money Bennett gets from selling eggplants: $3/eggplant  |
| 332 | 12 | 18 | ❌ | If Regina wrote 9 novels last year, and this is 3/4 of the number of novels she  |
| 333 | 63 | 16 | ❌ | Each child needs: - 2 hats - 4 scarves - 1 sweater - 1 pair of mittens - 2 pairs |
| 334 | 69 | 40 | ❌ | Each coin is worth $0.05. He has 8 * $0.05 = $0.40 from his quarters. He has 6 * |
| 335 | 9 | 5 | ❌ | Jess sees 2 * 17 = 34 red jellybeans. There are 60 - 17 - 34 = 5 blue jellybeans |
| 336 | 8 | 57 | ❌ | First find how many eggs are in five dozens: 5 dozens * 12 eggs/dozen = 60 eggs. |
| 337 | 18 | 18 | ✅ | If there are twice the number of boys as girls in the park, and there are 6 girl |
| 338 | 85 | 40 | ❌ | Jayden initially has $70 and his sister gives him $90 so now he has a total of $ |
| 339 | 7 | 7 | ✅ | There are 27 students and each student needs two glue sticks, for a total of 27* |
| 340 | 140 | 130 | ❌ | On the first day, Mr. Julius planted 2 * 20 = 40 Lodgepole Pines. He planted 40  |
| 341 | 28 | 36 | ❌ | First find how many birds were scared away from the backyard: 12 birds / 3 = 4 b |
| 342 | 10 | 10 | ✅ | If Aiden makes 16 meatballs per pound of ground beef, and he wants to make 80 me |
| 343 | 110 | 110 | ✅ | Anis rowed 50 x (1 + 1/5) = 60 miles. Together, they rowed 50 + 60 = 110 miles.  |
| 344 | 30 | 30 | ✅ | It will take 12 minutes x 6 paintings = 72 minutes for the coats to dry. The tim |
| 345 | 20 | 10 | ❌ | Let's assume that Sam has x feet of fence. Harry will get 60 feet more than Sam, |
| 346 | 25 | 25 | ✅ | Henry traveled 60 - 20 = 40 miles before his second stop. Thus, Henry traveled 4 |
| 347 | 1600 | 4 | ❌ | First find how many sticks Frederick can get from each 2 x 4 piece of wood: 200  |
| 348 | 240 | 80 | ❌ | Mattison’s house is 60 * 2 = 120 inches high. The first floor is 120 / 3 = 40 in |
| 349 | 10 | -75 | ❌ | If Jack drove 75 mph for 3 hours he traveled 75*3 = 225 miles. He spent 6+6=12 h |
| 350 | 8400 | -6000 | ❌ | At Job A, Nick makes $15/hour * 2000 hours = $30,000 per year. At Job B, Nick ma |
| 351 | 6 | -12 | ❌ | Dasha spends 9*2=18 dollars. David spends 18-30=-12 dollars. Therefore, the answ |
| 352 | 9360 | 1208 | ❌ | Sylvie’s salary increased by 10% annually over five years. So, her salary increa |
| 353 | 803 | 730 | ❌ | Karan borrowed $3,650 for five months at an interest rate of 10%. So each month  |
| 354 | 48 | 48 | ✅ | The dogs have 4 legs each, so that's 5 x 4 = 20 legs. The cats have 4 legs each, |
| 355 | 180 | 135 | ❌ | She needs 12 dinner plates so that costs 12 * $6 = $72.00 She also needs 12 sala |
| 356 | 2450 | 35 | ❌ | Each mamba has 70/2 = 35 spots. So together, each mamba has 35+35 = 70 spots. Th |
| 357 | 80 | 26 | ❌ | Let x represent the total number of students. Three-fourths of the students woul |
| 358 | 15 | 4 | ❌ | Each person gets 20/8=2.5 ounces of cake. So the remaining cake weighs 20-2*8 =  |
| 359 | 10 | 5 | ❌ | First find how many pages Mike needs to read per day: 200 pages / 30 days = 6.67 |
| 360 | 19 | 19 | ✅ | On Monday, Wednesday and Friday, Sam ran a total of 3 x 3 = 9 miles. On Tuesday  |
| 361 | 60 | 24 | ❌ | Pam rode the roller coaster 2 times *6 tickets per ride = 12 tickets. Fred rode  |
| 362 | 8000 | 8000 | ✅ | Let x be the number of copies Marilyn sold. Harald sold 10x copies since he sold |
| 363 | 1 | 194 | ❌ | First find the total number of smores needed: 8 smores/sleeve * 3 sleeves = 24 s |
| 364 | 54 | 19 | ❌ | Chase rides his bike 3*5=15 times a week. Rider rides his bike 2*2 = 4 times a w |
| 365 | 210 | 90 | ❌ | In the first 30 seconds, 20 kernels popped. In the next 30 seconds, 3*20=60 kern |
| 366 | 14 | 1 | ❌ | Each pizza has 8 / 8 = 1 slice. There are 7 * 3 = 21 slices in total. So each pe |
| 367 | 21 | 29 | ❌ | Let's say the first monkey eats x bananas per day. Then the second monkey eats 4 |
| 368 | 104 | 120 | ❌ | He saved up $12 x 12 = $144. For the first year, he bought 1 + 2 + 4 = 7 games.  |
| 369 | 6400 | 9000 | ❌ | First find how many sapphire rubies Jenna gets for trading 3 sapphires: 3 sapphi |
| 370 | 5 | 106 | ❌ | First find the total number of seats not used: 6000 seats - 950 seats - 300 seat |
| 371 | 80 | 24980 | ❌ | There were 100 burritos per student x 50 students = 5000 burritos. If each stude |
| 372 | 5 | 21 | ❌ | First find how much money Shawnda charged for each person's bicycle tire: $25 /  |
| 373 | 55 | 3025 | ❌ | A whirligig spins at 5 * 5 = 25 times faster than a thingamabob. Since a thingam |
| 374 | 8 | 8 | ✅ | Let x be the number of chocolate bars in one box. So, there were 3.5 + 4.5 = 8 b |
| 375 | 750 | 2000 | ❌ | Teddy placed 2 * 500 = 1000 pieces from the first puzzle. He also placed 500 + 5 |
| 376 | 350 | 700 | ❌ | He now makes $200 a day because 500 x .4 = 200 That means he saves $100 a day be |
| 377 | 6 | 6 | ✅ | One third of the total unicorns are in Scotland so that's 27 / 3 = 9 unicorns. T |
| 378 | 16 | 4 | ❌ | Joey lost 8/4 = 2 pounds per week. So Sandy needs 4*2 = 8 pounds to lose the sam |
| 379 | 56 | 38 | ❌ | Twice the number of players playing volleyball is 2*6=12. In total, there are 6+ |
| 380 | 50 | 46 | ❌ | On the second day, they sold 14 x 2 = 28 pairs. So by the end of the weekend, th |
| 381 | 8 | 8 | ✅ | If there are 20 pencils in the box and 4 pencils are missing, then there are 20  |
| 382 | 5600 | 5600 | ✅ | First find the total cost of the items Gabriel wants to buy: $10000 + $800 = $10 |
| 383 | 3 | 3 | ✅ | If a robe needs 2 bolts of blue fiber, then it also needs 1/2*2=1 bolt of white  |
| 384 | 975 | 525 | ❌ | The interest accrued over 3 years is (3/4) * 300 = $225. After 3 years Brenda wi |
| 385 | 50 | 60 | ❌ | Chris took 2 hours (60 minutes) to mow the lawn at the normal speed. If he mowed |
| 386 | 348 | 150 | ❌ | There are 2*60 = 120 cats. If there are 2 cats per dog, there are 60/2 = 30 dogs |
| 387 | 80 | 80 | ✅ | Let's assume that Brad gets x amount of money. Since Jeff gets 4 times as much a |
| 388 | 250 | 685 | ❌ | In total Jason has called 15 + 300 = 315 minutes. So he has 1000 - 315 = 685 min |
| 389 | 48 | 30 | ❌ | The final value is (20 + 20/2)/4 * (20 - 12) = 30. The answer is: 30. |
| 390 | 168 | 315 | ❌ | There were 700 * 100 / 40 = 175 more shells for the Alpha group. After that, the |
| 391 | 72000 | 28800 | ❌ | There are 60 minutes in an hour and 24 hours in a day, so there are 60*24 = 1440 |
| 392 | 840 | 840 | ✅ | The first news site has 20 * 28 = 560 articles. The second news site has 10 * 28 |
| 393 | 45 | 11 | ❌ | If Zack is twice the age of Adrian, he is 3 * 2 = 6 years older than Adrian. In  |
| 394 | 10 | 16 | ❌ | With the new box, each cycle uses up 12 * .2 = 2 pods. So you can do 1/2 = 5 cyc |
| 395 | 50 | 50 | ✅ | Argo had 200 toys. He gave 40 toys to Alyssa, so he now has 200 - 40 = 160 toys  |
| 396 | 5 | 5 | ✅ | If the plane travels at 400 miles per hour, then it will take 2000/400 = 5 hours |
| 397 | 7 | 8 | ❌ | The difference between the number of snails in the two aquariums is 32 - 4 = 28. |
| 398 | 43 | -235 | ❌ | There are 54*2=108 orange candies. There are 54/2=27 yellow candies. So there ar |
| 399 | 482 | 1508 | ❌ | The total cost is 200 + (300*30) + (4*30) + (6*2)= $1508. The answer is: 1508. |
| 400 | 130 | 52 | ❌ | First Elise writes the alphabet twice, which means she wrote 26 letters. Then sh |
| 401 | 500 | 1820 | ❌ | On the second day, there were 50 + 20 = 70 people at the beach. On the third day |
| 402 | 6 | 7 | ❌ | Let's say Matt has 2x carrot sticks. Half that amount would be (1/2)(2x) = x car |
| 403 | 7 | 14 | ❌ | If Jason had picked up and carried 2 balls, then Jordan would have carried twice |
| 404 | 10 | 21 | ❌ | He runs 18 mph because .5*3=1.5 That means he runs 27 mph because 3+18=21 The an |
| 405 | 91 | 0 | ❌ | He chopped down 10+2=12 trees per year So that leaves him with 50-12=38 trees Th |
| 406 | 6 | 27 | ❌ | If Vincent buys 9 flowers at 2 packages at $1 each, it will cost him $2 + $2 = $ |
| 407 | 168 | 79 | ❌ | If Andrea has 52 apples, she has 52+8 = 60 apples + bananas. Andrea has 60/2 = 3 |
| 408 | 20 | 112 | ❌ | Jack and his friends have a total weight of 8*14= 112kg. They will be able to li |
| 409 | 75 | 0 | ❌ | First, we find out how many bees flew away from the hive after the first 6 hours |
| 410 | 1198 | 965 | ❌ | First calculate the total cost of the picture frames without any extra charges:  |
| 411 | 78 | 13 | ❌ | There were a total of 15 x 60 = 900 sweets. So Anne kept 2 x 60 = 120 sweets for |
| 412 | 15 | 25 | ❌ | Paul still needs 63-8-40 = 25 more cupcakes. The answer is: 25. |
| 413 | 1240 | 6000 | ❌ | Each class plant 25*3=75 trees. In total, 75*40 = 3000 trees are planted. Theref |
| 414 | 50 | 27 | ❌ | If 40 minutes of her walk is spent on the beach, then she spends (40/60) * 3 = 2 |
| 415 | 800 | 80 | ❌ | First find how many calories Hannah burns while running: 600 calories/hour * 1 h |
| 416 | 8 | 4 | ❌ | Ferdinand has 6 / 2 = 3 trees. Harry has 3 + 5 = 8 trees. There are 6 - 8 = 4 fe |
| 417 | 4 | 3 | ❌ | After four weeks, Josh has 3 + 4 = 7 balls. When he loses the third ball, he can |
| 418 | 200 | 1920 | ❌ | She walks 40 minutes each day because 8 x 5 = 40. She walks 240 minutes at the o |
| 419 | 38 | 84 | ❌ | If four years ago, Adam was 4 + 4 = 8 years old. Eight years ago, Duncan and Ada |
| 420 | 3528 | 25440 | ❌ | First he took the boat out of the garage and let it sit there so that it would l |
| 421 | 31 | 39 | ❌ | First find the total value of the quarters: 32 quarters * $0.25/quarter = $8. Th |
| 422 | 230 | 440 | ❌ | At the discount, the car radiator cost him $400*.8=$320. After getting it discou |
| 423 | 75 | 75 | ✅ | First find how many trays the farmer can fill with 900 eggs: 900 eggs / 30 eggs/ |
| 424 | 54 | 91 | ❌ | Half as many points as Naomi was 68/2 = 34. Yuri scored 34 + 10 = 44 points. Bri |
| 425 | 5 | 5 | ✅ | Each part of the first cut is 100/4=25 cm. Each part of the second cut is 25/5=5 |
| 426 | 16 | 20 | ❌ | In the first half, team B scored 2 - 2 = 0 goals. In the second half, team A sco |
| 427 | 20 | 33 | ❌ | Kyle took 5 + 5 = 10 fries. Billy took 2 x 10 = 20 fries. Ginger gave Griffin 20 |
| 428 | 3 | 3 | ✅ | He paid $15 for his purchase because .5 + .5 + .5 + .75 = $2.75 There were $15 - |
| 429 | 60 | 60 | ✅ | There will be 50*1.2=60 students this year. The answer is: 60. |
| 430 | 2 | 5 | ❌ | He uses 4*3=12 ounces for the giant cubes. He uses 2*7=14 ounces for the medium  |
| 431 | 60 | 60 | ✅ | Sam needs 5 kg * $3/kg = $15 for 1 liter of juice. So he spends $15/liter * 4 li |
| 432 | 5 | 100 | ❌ | First find the total amount of straw used to feed the hamsters: 10 * 5 = 50 piec |
| 433 | 315 | 630 | ❌ | First find how many stitches it took for the collar: 900 stitches / 10 = 90 stit |
| 434 | 276 | 276 | ✅ | For the cars, he will use 4 * 57 = 228 wheels. For the motorcycles, he will use  |
| 435 | 25 | 25 | ✅ | There are 50 - 10 = 40 students who like to play other than music and basketball |
| 436 | 72 | 12 | ❌ | First, we need to calculate the initial cost of Maria’s shares of the stock. She |
| 437 | 120 | 75 | ❌ | If Arelia has $120, then Kassidy has 3/4*120 = $90. Together, Kassidy and Rayna  |
| 438 | 3 | 4 | ❌ | Let c be the number of cupcakes and b be the number of brownies. We know that c  |
| 439 | 1 | 74 | ❌ | Robyn has 120 emails because 60 x 2 = 120 Seven are work emails so there are 120 |
| 440 | 26 | 14 | ❌ | First find how much money Kyle saved by applying the discount to his original pr |
| 441 | 13 | 6 | ❌ | First find how much money Carlos earns from selling one lemon: $1.5 per lemon *  |
| 442 | 48 | 9 | ❌ | A kangaroo can cover a distance of 3 x 18 = 54 kilometers in 1 hour. Thus, a tur |
| 443 | 14 | 14 | ✅ | Jean initially had 30 - 2 = 28 lollipops left. She can fill 28 / 2 = 14 bags wit |
| 444 | 2 | 2 | ✅ | He made $80 x 1.25 = $100 per watermelon. So he sold each one for $100 / 50 = $2 |
| 445 | 21 | 21 | ✅ | Rachel had 23 + 42 = 65 cookies. After her brother ate some, there were 65 - 44  |
| 446 | 12 | 12 | ✅ | If Marissa spent $20 on the ball, then she has $80 - $20 = $60 remaining. Since  |
| 447 | 9 | 151 | ❌ | Jenny takes 1 x 5 = 5 bags of cookies each week. So, she takes 36 x 5 = 180 cook |
| 448 | 16 | 16 | ✅ | Joseph initially had 9 x 2 = 18 books. So, after losing two books, Joseph now ha |
| 449 | 160 | 4 | ❌ | First, we find out how long Carla had before the restart because she had to star |
| 450 | 2880000 | 240000 | ❌ | If the company hires 20 new employees monthly, it will hire 20 x 3 = 60 new empl |
| 451 | 6277 | 12770 | ❌ | There are 23786 - 8417 - 9092 = 6385 people who are not children. So there are 6 |
| 452 | 12 | 2 | ❌ | First find how much water 20-4=16 plants need in total: 16*0.5 = 8 cups. Then su |
| 453 | 82 | 44 | ❌ | He bought 38+6=44 sausages. Therefore, the answer is: 44. |
| 454 | 18 | 18 | ✅ | He goes up 4(3) + 6 = 18 floors. So he is on the 18th floor. The answer is: 18. |
| 455 | 50 | 240 | ❌ | Each CD costs $15 so they cost 15*10 = $150. They get them for 40% off so the pr |
| 456 | 11 | 17 | ❌ | Jack plays one game at a time. The two people that are significantly worse than  |
| 457 | 9 | 17 | ❌ | There are 42 adult + 15 baby = 57 people. Mia's restaurant has 8 chairs / person |
| 458 | 5 | 12 | ❌ | Let's say that initially there were x cars on the road. If some cars took the le |
| 459 | 9 | 51 | ❌ | He scored 8 points because .25*4=1 point more. In the next 20 minutes, he scored |
| 460 | 10 | 10 | ✅ | She has to write 15 x 1/3 = 5 pages. So, she still needs to write 15 - 5 = 10 pa |
| 461 | 120 | 120 | ✅ | Jack received 100*1/5=20 dollars from Sophia. So Jack has 100+20=$120 left. The  |
| 462 | 67 | 69 | ❌ | Albert's height = 36 inches. Anne is twice the size of Albert, so her height is  |
| 463 | 60 | 20 | ❌ | She gets 20*2=$40 from selling the cupcakes. She makes $40/40=1 per cookie. So s |
| 464 | 8 | 8 | ✅ | Alyana is 4-10 = 6 years old. If Anne is 2 years older than Alyana, she is 6+2 = |
| 465 | 1 | 1 | ✅ | A magazine cost 1/2*4=2. The price of a pen is 2-1=$1. The answer is: 1. |
| 466 | 4 | 9 | ❌ | On the first day, when it was 45 degrees, the difference between the temperature |
| 467 | 28 | 28 | ✅ | If Marcia has 2 more pets than Cindy, then Marcia has 4+2=6 pets. Together, Marc |
| 468 | 80 | 80 | ✅ | Her total earnings for the week was 200 - 420 = -$220. She needs to take out $30 |
| 469 | 32 | 32 | ✅ | If Quinn is 30, then Jane is 30 - 3 = 27. And if Jane is 27, then Trent is 27 +  |
| 470 | 17 | 13 | ❌ | There are 20*3=60 students in the school. There are 60-15-12 = 33 girls in the f |
| 471 | 1520 | 28 | ❌ | Julieta’s tip was $20 x 0.1 = $2. Julieta got a total of $2 + $20 = $22 from her |
| 472 | 12 | 10 | ❌ | One pencil = $1.20 One eraser = $0.30 Eight pens = (8 x $1.20) + ($0.30) = $9.60 |
| 473 | 2050 | 100 | ❌ | He spends 6*100=$600 on the suits. It costs him 5*$100=$500 on the tickets. That |
| 474 | 5 | 8 | ❌ | George spent $5 - 1/10 * $5 = $5 - $1 = $4. He bought 3 bags of chips at 40 cent |
| 475 | 2 | 1 | ❌ | If the phone is currently at 60%, it means that only 40% remains. So, we need to |
| 476 | 55 | 75 | ❌ | First find total crates sold over Saturday and Sunday: 15 + 12 + 18 = 45 crates  |
| 477 | 128 | 76 | ❌ | First find how many glasses Jeff had originally: 8 dozens * 12 glasses/dozen = 9 |
| 478 | 5 | 19 | ❌ | If Sarah spent $300 on the books, then each book cost $300/4 = $75. Since each b |
| 479 | 15 | 15 | ✅ | First find how many jello cups Greg needs to make for his son's guests: 30 kids  |
| 480 | 32 | 16 | ❌ | One family-size pizza costs 1 x 8 = $8. So, Sally spends 8 + 8 = $16 in all. The |
| 481 | 84 | 72 | ❌ | Anne went down the slide 30 - (30 * .30) = 18 times. Robert went down the slide  |
| 482 | 120 | 100 | ❌ | Last Monday Linus bought a mobile device for $20. He sold it for twice that amou |
| 483 | 75 | 75 | ✅ | Twice as many students need size small as extra small = 2 * 11 = 22 Four less th |
| 484 | 15 | 15 | ✅ | If there are 7 more dahlias than roses, then there are 7 + 4 = 11 dahlias. In th |
| 485 | 30 | 30 | ✅ | Since Martha is 4 years younger than Mike, she is 24-4 = 20 years old. Since Jef |
| 486 | 225 | 63 | ❌ | Each student needs 45/4 = 11.25 minutes. There are 3 groups of 4 students so 3 x |
| 487 | 17 | 8 | ❌ | There are 4 kids on varsity because 3 x 1/4 = 3/4. There are 6 kids not on varsi |
| 488 | 19 | -1 | ❌ | Craig spent 6*2=12 dollars on his squirt guns. He also spent 3*3=$9 on his water |
| 489 | 14 | 9 | ❌ | At the first stop, there were 1 + 5 = 6 people on the bus. Then there were 6 - 7 |
| 490 | 120 | 96 | ❌ | Judy bought 12 x 2 = 24 plates. Judy bought 40 + 20 = 60 other items. Judy bough |
| 491 | 90000 | 120000 | ❌ | Let's say that the amount of the lot costs x dollars. Since the house cost three |
| 492 | 14 | 14 | ✅ | First find how many boxes of popsicle sticks Bob needs: 56 sticks / 8 sticks/box |
| 493 | 12 | 18 | ❌ | One candle costs $3/5=$6 each because 5 x 6 = 30 He needs two packs so he spends |
| 494 | 15 | 45 | ❌ | If cleaning all 90 rooms would take 1800 minutes (since there are 60 minutes per |
| 495 | 8 | 8 | ✅ | Let x = number of coffee mugs. Four times the number of coffee mugs is 4x Eight  |
| 496 | 17 | 24 | ❌ | Her history book weighs 3 * 2 = 6 pounds. Her English book weighs 6 + 3 = 9 poun |
| 497 | 45 | 17 | ❌ | She collects 30/2 = 15 kilograms of milk per day. She gets 15+2 = 17 kilograms o |
| 498 | 280 | 395 | ❌ | First find how many hours Jen works in a month: 7.5 hours/day * 6 days/week * 4  |
| 499 | 35 | 70 | ❌ | First find how much the tip will cost: $50 x 20/100 = $10 Then add that amount t |
| 500 | 5 | 25 | ❌ | Let's assume that Bahati scored x points. Azibo scored x+20 points. Dinar scored |
| 501 | 13 | 8 | ❌ | First find how long the second song is: 60/100*5 = 3 minutes Then add that to th |
| 502 | 14 | 14 | ✅ | Darren caught 17 - 6 = 11 fish. Tommy caught 11 + 3 = 14 fish. The answer is: 14 |
| 503 | 60 | 60 | ✅ | Each extra phone costs $20 / 2 = $10. So, the total cost of the whole phone plan |
| 504 | 5 | 15 | ❌ | First find how many points Joey has after he adds his score to his original scor |
| 505 | 6 | 3 | ❌ | He needs to read 15+4*8=53 pages. He will be left with 40-53=-13 pages. On avera |
| 506 | 6000 | 13875 | ❌ | Wanda filled 1/4*18000 = 4500 gallons. Ms. B filled 3/4 * 4500 = 3375 gallons. O |
| 507 | 30 | 39 | ❌ | First find the total amount of beef tips eaten by the four players: 4 players *  |
| 508 | 44 | 66 | ❌ | If 40% of the castles are ruins, then 100 - 40 = 60% of the castles are inhabite |
| 509 | 72 | 33 | ❌ | Since 1 Mb = 1000 Kb, 1 Mb per second = 1/1000 Mb per second. Therefore, 20 Mb/s |
| 510 | 6 | 1 | ❌ | In total, Bubbles has 3 + 5 + 2 + 4 = 14 stuffed animals. To get 30% of all her  |
| 511 | 1 | 17 | ❌ | First find the total cost of the items Dora wants to buy: 6 + 11 + 2 = 19 Then s |
| 512 | 7 | 7 | ✅ | On Saturday, Joey played 2 * 2 = 4 matches. So in total, Joey played 2 + 4 + 1 = |
| 513 | 11 | 14 | ❌ | Jasper has 2*4=8 blue paint tubes. Jasper also has 3*2=6 yellow paint tubes. Jas |
| 514 | 20 | 20 | ✅ | Half of the hats are red so there are 100 / 2 = 50 red hats. This leaves 100 - 5 |
| 515 | 180 | 180 | ✅ | For the time that Carlos spent renting his canoe, he paid 30*3=90 dollars. Benji |
| 516 | 50 | 50 | ✅ | If the distance around the Earth is 40,000 kilometers, then half of that distanc |
| 517 | 35 | 210 | ❌ | Sofie walks 10*2=20 miles a day. Brian walks 10*3=30 miles a day. So in seven da |
| 518 | 16 | 320 | ❌ | She gets 60/3=20 servings each day. So after 60 days, she'll get 20*60=1200 serv |
| 519 | 23 | 33 | ❌ | First find the number of green cars: 12 red cars - 2 = 10 green cars Then multip |
| 520 | 70 | 62 | ❌ | First find the total sum of all Sally's quiz scores: 50 + 80 + 80 + 60 + 40 + 90 |
| 521 | 1375 | 1375 | ✅ | He spent $5*50=$250 on newspaper ads. And he spent $75*15=$1125 on television ad |
| 522 | 105 | 80 | ❌ | Each quarter is worth 25 cents, so that means the quarters make 45 cents. Since  |
| 523 | 298 | 178 | ❌ | Max has 12 + 68 = 80 cookies. Since Katarina has 5 less cookies than Max, she ha |
| 524 | 52 | 14 | ❌ | Buford writes 12 checks per year because there are 12 months in a year. Out of t |
| 525 | 81 | 324 | ❌ | He saves 12 * 900 = 10800 watt-hours per day. So, he saved 10800 / 1000 = 10.8 k |
| 526 | 91 | 543 | ❌ | For the first part, we have that 1 hour (60 minutes) / 20 = .3 hours. So after w |
| 527 | 21 | 28 | ❌ | He eats 2*6=12 tomato daily. A week has 7 days so he eats 12*7=84 tomato every w |
| 528 | 64800 | 14 | ❌ | First find how many widgets the WidgetMaster 9000 can produce in one day: 45 wid |
| 529 | 6 | 36 | ❌ | Each container held 3 gallons because 15 / 5 = 3. Josey used 9 gallons for her l |
| 530 | 100 | 100 | ✅ | Each acre produces 5*2 = 10 barrels of wine. So, the whole farm will produce 10* |
| 531 | 25 | 25 | ✅ | Let x be Carver's son's age. Twice Carver's age is 2x. If Carver is 5 years youn |
| 532 | 16 | 18 | ❌ | First find the total weight of John's luggage and children: 1200 + 250 + (75*2)  |
| 533 | 5 | 45 | ❌ | First find the total number of eggs Sandra will need for the flans: 9 eggs/neigh |
| 534 | 150 | 420 | ❌ | First find how many total bird feeds Lillian bought: 3 + 3 = 6. Then multiply th |
| 535 | 31 | 7 | ❌ | Oscar started with 24 lollipops. He ate 2 so now he has 24 - 2 = 22 left. He giv |
| 536 | 3430 | 360 | ❌ | The total number of people infected after one day is 10*6=60. After two days the |
| 537 | 40 | 40 | ✅ | Ashley brought 5 x 10 = 50 oranges. So Brianne brought 50 + 20 = 70 oranges. The |
| 538 | 3 | 3 | ✅ | Last night, James slept 2 / 3 * 9 = 6 hours. So, Harry slept 9 - 6 = 3 more hour |
| 539 | 70 | 70 | ✅ | 30/100 * $100 = $30 off. $100 - $30 = $70 The answer is: 70. |
| 540 | 145 | 555 | ❌ | The first train leaves the station in 10 + 20 = 30 minutes. The second train sta |
| 541 | 280 | 180 | ❌ | Out of the 1000 students initially in the hall, 30% of them left through exit A, |
| 542 | 8 | 8 | ✅ | Deepa has seen each movie three times, so she has seen 9*3=27 movies. She has sp |
| 543 | 21 | 21 | ✅ | Ann's brother is 9 x 2 = 18 years old. So, he will be 18 + 3 = 21 years old in 3 |
| 544 | 11 | 9 | ❌ | The soup costs 25/100 * $4 = $1. The total cost of the soup, bagel, and cake is  |
| 545 | 14 | 4 | ❌ | On low, the air conditioner cools the room by 2 degrees per hour, so over 3 hour |
| 546 | 24 | 44 | ❌ | There were 10 meatball sub sandwiches * 4 meatballs = 40 meatballs. Sidney order |
| 547 | 25 | 25 | ✅ | If Ryan planted 2 flowers per day and 15 days passed, he would plant 2 x 15 = 30 |
| 548 | 842 | 522 | ❌ | First find the total number of bottles of apple juice the teacher gave out: 9 co |
| 549 | 52 | 62 | ❌ | The difference between Jared's typing speed after each lesson is 15 words per mi |
| 550 | 245 | 23500 | ❌ | He has 700*2 = 1400 big coupons. So he spends 1400*15=21000 cents on big coupons |
| 551 | 24 | 6 | ❌ | At first, there were 18/3 = 6 pink flamingos. So at the end of Saturday, there w |
| 552 | 300 | 300 | ✅ | Sophia could travel 100 miles / 4 gallons = 25 miles per gallon. Therefore, she  |
| 553 | 12 | 7 | ❌ | Miss Susan has 5 + 8 = 13 students who are good at either math or english. Thus, |
| 554 | 9 | 9 | ✅ | Let x be the number of books that Sally has. If Janey has 3 more than twice the  |
| 555 | 11 | 132 | ❌ | Carla needs 2 cups of flour for each dozen cookies. So she will need 2 x 36 = 72 |
| 556 | 227 | 173 | ❌ | First find the total time needed to string the synthetics: 3 racquets * 15 minut |
| 557 | 4 | 2 | ❌ | The first two dogs together need 2 * 1 = 2 miles. The third dog needs 3 miles. S |
| 558 | 460 | 880 | ❌ | First find the amount Eliza earns from her regular work: 40 hours * $10/hour = $ |
| 559 | 2 | 2 | ✅ | Juice Box B costs $4 + $5 = $9. Juice Box C costs $4 + $7 = $11. Juice Box C cos |
| 560 | 36 | 36 | ✅ | First find the amount of the discount: $30 x 40/100 = $12. Then subtract the dis |
| 561 | 52 | 52 | ✅ | Kimberly bought 8 x 11 = 88 cans of cat food. She also bought 6 x 6 = 36 cans of |
| 562 | 35 | 40 | ❌ | To determine how long it takes Richard to set off all the soda fountains, we nee |
| 563 | 80 | 40 | ❌ | Celinda baked 2 * 30 = 60 cookies. Together, they ate 10 + 10 = 20 cookies. They |
| 564 | 600 | 1800 | ❌ | If Katina takes out $100 per month and there are 12 months in a year, then she w |
| 565 | 144 | 168 | ❌ | In the second year, they will build 3 * 12 = 36 homes. So in total, they will bu |
| 566 | 25 | 80 | ❌ | Since there are two adults and four children, they will have a total of 6 people |
| 567 | 32 | 8 | ❌ | On its first bounce, the ball reaches a height of (2/3)*24=16 feet. On its secon |
| 568 | 1800 | 5400 | ❌ | He bought 1+2+1=3 phones So it was 3*$600=$1800 per phone It cost $1800*3=$5400  |
| 569 | 650 | 670 | ❌ | If Craig's brother has spelled words with a total count of letters 50 more than  |
| 570 | 40000 | 90000 | ❌ | His loan was 40/100*250000=$100000 He used 60/100*100000=$60000 for debt So he h |
| 571 | 125 | 2910 | ❌ | If the merchant chooses the jewelry plan and it goes up 2.5%, his profit would i |
| 572 | 360 | 144 | ❌ | If 40% of the students are boys, that means 100 - 40 = 60% are girls. So, there  |
| 573 | 260 | 260 | ✅ | If Seattle has 20 sheep, then Charleston has 4*20 = 80 sheep. And if Charleston  |
| 574 | 350 | 350 | ✅ | Luke needs to buy 5*2=10 inches of fabric. So he needs to buy 10*35=350 inches o |
| 575 | 16 | 16 | ✅ | There are 52*0.5=26 girls. There are 52*0.5=26 boys. There are 52-26=26 females  |
| 576 | 7500 | 150 | ❌ | Publisher A pays Mark $5/100 = $0.05$cents per sentence. Since he edits 1000 sen |
| 577 | 120 | 130 | ❌ | Frank starts with 200 pairs of shoes. On Monday, he buys 5 pairs of shoes. On We |
| 578 | 31800 | 390 | ❌ | He drives 60 miles each way because 30*2=60 So he drives 120 miles each day. Tha |
| 579 | 300 | 290 | ❌ | First find how much Suzanne earned from selling cookies: 80 cookies * $1/cookie  |
| 580 | 3 | 3 | ✅ | Out of the 50 employees, 20/100 * 50 = 10 manage the company. Out of these 10 ma |
| 581 | 40 | 5 | ❌ | Peter's locker is 5*4 = 20 cubic inches. Timothy's locker is 20/4 = 5 cubic inch |
| 582 | 2100 | 1300 | ❌ | In the first month, Ariadne sold 2*400 = $800 worth of green hats. Therefore, in |
| 583 | 216 | 24 | ❌ | First find the total volume of the hole in cubic feet: 6 ft x 4 ft x 3 ft = 72 c |
| 584 | 193 | 611 | ❌ | In total James has delivered 600 + 209 = 809 newspapers. He has delivered 198 +  |
| 585 | 34 | 40 | ❌ | 6 x (2/3) = 4 birds. 4 + 8 + 8*3 = 40 birds. The answer is: 40. |
| 586 | 87 | 63 | ❌ | First find how many total outfits Laurel had from her friends: 24 outfits + 24 o |
| 587 | 428 | 447 | ❌ | Nikita scored 278 / 2 + 11 = 169 points. So, together they had 278 + 169 = 447 p |
| 588 | 4 | 4 | ✅ | There are 16 / 2 = 8 golf balls. Half of the golf balls are blue so there are 8  |
| 589 | 9 | 16 | ❌ | First find how many cards have the letter A: 80 cards * 2/5 = 32 cards. Then sub |
| 590 | 205 | 187 | ❌ | Patty charged 40 + (35 * 2.25) = $187. The answer is: 187. |
| 591 | 20 | 10 | ❌ | Robert has 30+10 = 40 cherries. They all have 60-40 = 20 cherries left over. The |
| 592 | 12 | 6 | ❌ | First find how many inches of ribbon Monica will need to tie each gift bow: 144  |
| 593 | 270 | 270 | ✅ | Maddison starts off with 5*50 = 250 marbles. After getting 20 more from her frie |
| 594 | 24 | 12 | ❌ | Each teacher takes 2 lessons every day so they use the whiteboard for 2*2=4 days |
| 595 | 2 | 12 | ❌ | Rissa was given $5 x 3 = $15. After giving some money to their little sister, th |
| 596 | 2 | 2 | ✅ | First find how many minutes it takes to go down the biggest slide: 300 feet / 60 |
| 597 | 80 | 1000 | ❌ | First find twice the first friend's guess: 20 + 20 = 40 Then add that to the sec |
| 598 | 15 | 4 | ❌ | Let the number of electric poles be x. Then, the number of electric wires needed |
| 599 | 50 | 36 | ❌ | From Monday to Wednesday, Lucy sold 4 * 3 = 12 pieces of apples. She sold 12 + 1 |
| 600 | 860 | 245 | ❌ | For the first two weeks, Micheal rode his bike 5 times a week and each time he m |
| 601 | 320 | 720 | ❌ | Ethan's original cost for the wallpaper was $400. DIY saves him 20%, which means |
| 602 | 240 | 240 | ✅ | There are 4 weeks in a month, so Miguel uses 4 x 2 = 8 pads of paper per month.  |
| 603 | 90 | 280 | ❌ | She got $10 x 10 = $100 from selling the 2 x 4 x 10 boards. She got $16 x 5 = $8 |
| 604 | 50 | 90 | ❌ | Let's assume that Elvis saves $x per day in the first half of the month. Then, h |
| 605 | 260 | 165 | ❌ | The blue beetle weighs 50 - 10 = 40 pounds. The green toad weighs 50 - 20 = 30 p |
| 606 | 2 | 3 | ❌ | Lana divides the 27 cups equally into 3 rows so that there are 9 cups per row. I |
| 607 | 39 | 33 | ❌ | Chris completed 12 x 3 = 36 problems on Tuesday. There were 100 - 12 - 36 = 44 p |
| 608 | 11 | 43 | ❌ | First find the total cost of the book before any shipping costs: $32 + ($32 * .3 |
| 609 | 826 | 8160 | ❌ | First, we need to find the volume of the pool using the formula length x width x |
| 610 | 6 | 5 | ❌ | Kate has 6 cheese pizzas x 6 slices = 36 cheese pizza slices. She also has 6 pep |
| 611 | 100 | 29 | ❌ | He needs to spend 6 hours total - 20 min + 40 min + 20 min + 3 h = 58 min. There |
| 612 | 70 | 80 | ❌ | He sold 10 - 10 = 0 goats from Farm X. He also sold 2 x 10 = 20 goats from Farm  |
| 613 | 272 | 112 | ❌ | Kimo spends 3*1=3 hours on his first three days. He spends 2*2=4 hours on his se |
| 614 | 60 | 6 | ❌ | First find the total number of branches Felix climbed while climbing (105 / 0.25 |
| 615 | 60 | -16 | ❌ | Since each level has half the square footage of the level above it, then the tot |
| 616 | 5 | 9 | ❌ | He sleeps 10*2=20 hours a week. So he has 24-20=4 hours of sleep remaining. He s |
| 617 | 3 | 5 | ❌ | First find how many pounds of almonds are in the mixture: 1/2 pound / $10 per po |
| 618 | 26 | 30 | ❌ | First find how many gifts the first two friends need: 5 gifts/friend * 2 friends |
| 619 | 15 | 29 | ❌ | If Andy drives to work every day and it takes him 25 minutes to drive to work, h |
| 620 | 60 | 45 | ❌ | On Tuesday and Wednesday combined, she removed 2 * 39 = 76 bugs. So she removed  |
| 621 | 17500 | 100 | ❌ | He made 25000*.3=$7500 So he got 25000*.1=.25*1000=250 dollars from commissions  |
| 622 | 8 | 11 | ❌ | Before eating, there were 3 x 2 = 6 slices of cake. After Mara took some from th |
| 623 | 48 | 30 | ❌ | If Mia is 40 years old, then her sister is 16 - 40 = 20 years old. Their combine |
| 624 | 50 | 100 | ❌ | First find how many minutes the class can be cleaned in one day: 60 minutes/day  |
| 625 | 450 | 50 | ❌ | It takes 2 x 75 = 150 medium-sized jelly beans. So, he can eat 150/3 = 50 small  |
| 626 | 6 | 4 | ❌ | DJ's total amount of guns = 8 guns. Nick's total amount of guns = 10 guns. RJ's  |
| 627 | 114200 | 263200 | ❌ | He gets his award in January so it's worth $5000. He earns a raise for the rest  |
| 628 | 131250 | 195 | ❌ | Let's first calculate how many of each type of bike were sold. Half of the bikes |
| 629 | 159 | 81 | ❌ | Twice the amount that each goat needs is 5*2=10 pounds. Each sheep needs 3-3=0 p |
| 630 | 3 | 9 | ❌ | Frankie watched 60 minutes from Monday to Thursday because 2 x 30 = 60. He watch |
| 631 | 120 | 55 | ❌ | The zoo has 20 x 2 = 40 cows. The farm has 30 / 2 = 15 sheep. Together, they hav |
| 632 | 93 | 90 | ❌ | There are 30 days in May. So Jen spends 30 x $1 = $30 on fish per day. In total, |
| 633 | 25 | 499975 | ❌ | There will be 49500 candles that do not smell like wet dog. So there will be 500 |
| 634 | 17 | 9 | ❌ | If it takes Wayne four minutes to be driven to the premiere, then it will take h |
| 635 | 90 | 180 | ❌ | Celine had 120 - 90 = 30 liters remaining. Rainwater collected by Celine is 30 * |
| 636 | 120 | 160 | ❌ | Pancho walks 20*7=140 miles a week. Pancho walks 20-10=10 miles a weekend. Panch |
| 637 | 2 | 7 | ❌ | Ryan earned $6 x 3 = $18 from doing his chores. He spent $1.25 x 4 = $5 on ice c |
| 638 | 23 | 17 | ❌ | He started with 9 oatmeal cookies, gave away 1 for lunch so that leaves him with |
| 639 | 66 | 236 | ❌ | On the first stop there were 48 - 8 = 40 passengers remaining. There were 5 * 8  |
| 640 | 34 | 345 | ❌ | Charisma meditates for 60 minutes a day because 1 hour x .5 = 5 minutes. She spe |
| 641 | 90 | 90 | ✅ | Her sister gave her 10+20 = 30 beads. She got twice as many beads from her frien |
| 642 | 2 | 1 | ❌ | Let x be the price per pack of strawberry gum. Since Suzie bought 4 packs of gum |
| 643 | 95200 | 57600 | ❌ | If Adrien’s total salary was 30 percent higher than Lylah’s, and Lylah’s salary  |
| 644 | 221 | 218 | ❌ | Tommy makes $3 x 43 = $126 from selling brownies. He also makes $4 x 23 = $92 fr |
| 645 | 31 | 82 | ❌ | They got 43/20 = 2.15 coupons. So they were able to get a reward of 2 x $1 = $2. |
| 646 | 12 | 14 | ❌ | First find how many guests come to John's birthday party: 20 guests + 1 original |
| 647 | 2 | 1 | ❌ | There are 4 adults * 5 bars/adult = 20 chocolate bars. So there are 20 bars / 8  |
| 648 | 120 | 120 | ✅ | If each of the 40 restaurants receives 2 pounds of carrots, then a total of 40 * |
| 649 | 70 | 70 | ✅ | Let's say there are G gold coins and S silver coins. We know that G + S = 110 (t |
| 650 | 6 | 3 | ❌ | Each car tire rotates 725 times per mile. Jeremy drives 400 miles a month so tha |
| 651 | 15 | 15 | ✅ | If Eve was left with 1/4 of the original amount, then she gave away 3/4 of the o |
| 652 | 11 | 11 | ✅ | It takes them 6 + 13 = 19 minutes to get to the fire station. They have 30 - 19  |
| 653 | 8 | 10 | ❌ | William spent 5*1.5=$7.5 He also spent .5*$5=$2.5 So he spent 7.5+2.5=10 The ans |
| 654 | 16 | 848 | ❌ | If the dozen of flowers costs $15, then one rose costs $15/12 = $1.25. So, 15 ro |
| 655 | 10 | 80 | ❌ | If 25% of the riders fall off the wave, then 75% remain unharmed. This means 25/ |
| 656 | 280 | 280 | ✅ | There are 8 x 7 = 56 children in each row. So, there are a total of 56 x 5 = 280 |
| 657 | 140000 | 1000000 | ❌ | In one week, the company will produce 5000 bars * $2/bar = $10000 worth of choco |
| 658 | 76 | 76 | ✅ | On Tuesday Rose bought 3 * 4 = 12 cakes. On Wednesday she bought 5 * 12 = 60 cak |
| 659 | 120 | 120 | ✅ | First find the total distance Hannah runs: 8 laps * 0.25 km/lap = 2 km Then mult |
| 660 | 39 | 34 | ❌ | Let's start by determining how much each of the three boys received initially. S |

## Merge & Push to HuggingFace

In [43]:
model_card = f"""---
tags:
- math
- gsm8k
- sft
- lora
- qwen2.5
datasets:
- openai/gsm8k
base_model: {config["model_name"]}
---

# {config["hf_repo"].split("/")[-1]}

Fine-tuned **{config["model_name"]}** on [GSM8K](https://huggingface.co/datasets/openai/gsm8k) for math reasoning with a structured answer format.

## Training Details

| Parameter | Value |
|---|---|
| Base model | `{config["model_name"]}` |
| Method | SFT + LoRA |
| LoRA r / alpha | {config["lora_r"]} / {config["lora_alpha"]} |
| LoRA targets | all-linear |
| Epochs | {config["epochs"]} |
| Learning rate | {config["lr"]} |
| Batch size | {config["batch_size"]} × {config["grad_accum"]} (grad accum) |
| Training examples | {config["train_size"]} |
| Precision | bf16 |

## Answer Format

The model is trained to end responses with:
```
The answer is: {{number}}.
```

## System Prompt

```
{SYSTEM_PROMPT}
```
"""

from huggingface_hub import ModelCard, create_repo

create_repo(config["hf_repo"], exist_ok=True)

ModelCard(model_card).push_to_hub(config["hf_repo"])
model.push_to_hub(config["hf_repo"])
tokenizer.push_to_hub(config["hf_repo"])
print(f"Pushed to https://huggingface.co/{config['hf_repo']}")


README.md:   0%|          | 0.00/900 [00:00<?, ?B/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...nwrkgxh/model.safetensors:   5%|5         | 50.3MB /  988MB            

README.md:   0%|          | 0.00/906 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpeg26zut5/tokenizer.json:  73%|#######2  | 8.30MB / 11.4MB            

Pushed to https://huggingface.co/tripathysagar/Qwen2.5-0.5B-GSM8K-SFT
